In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('transformed_df.csv')
df.drop(['Question', 'Unnamed: 24'], axis=1, inplace=True)
df.head()


,Number of Sources,Trust Score,chatGPT,copilot,gemini,llama,perplexity,Economics & Commerce,Entertainment,Food & Cooking,...,y20india.in,yahoo.com,yale.edu,yaleclimateconnections.org,yellowbrick.co,yourdictionary.com,yourstory.com,youtube.com,zavvi.com,zerohanger.com
0,2,10,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,9,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,11,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,11,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,9,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df.drop('Trust Score', axis=1)
y = df['Trust Score']


kf_2 = KFold(n_splits =5, shuffle=True, random_state=42)
mae_scores = []

for train_index, val_index in kf_2.split(X):
    # Split the data
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    # Convert data to DMatrix (Native XGBoost format)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    # Define parameters for XGBoost
    params = {
        'objective': 'reg:squarederror',  # Regression objective
        'learning_rate': 0.05,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
    }

    # Train the model with early stopping
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=[(dval, 'validation')],
        early_stopping_rounds=10,
        verbose_eval=False,
    )

    # Make predictions
    y_pred = model.predict(dval)

    # Calculate MAE
    mae = mean_absolute_error(y_val, y_pred)
    mae_scores.append(mae)
    print(f"Fold MAE: {mae:.2f}")

# Compute the mean MAE across folds
mean_mae = np.mean(mae_scores)
print(f"Mean MAE across folds: {mean_mae:.2f}")

Fold MAE: 0.28
Fold MAE: 0.27
Fold MAE: 0.28
Fold MAE: 0.32
Fold MAE: 0.28
Mean MAE across folds: 0.29
